In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
import torch
import torch.nn as nn
from transformers import AutoModel, TrainingArguments, Trainer, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'Tadawul_sa'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,Precision,Recall,F1\n')


df = pd.read_csv('Tadawul_Sentiment_Emotions_Analysis.csv', encoding='utf-8', engine='python', sep='\t')



df.fillna('', inplace=True)  
display(len(df))


display(df.columns)
display(len(df))
display(df[:4])



classes = set(df['sentiment'].values)
display(classes)

c = df['sentiment'].value_counts()
display(c)

df['sentiment'] = df['sentiment'].astype('category')
df['label'] = df['sentiment'].cat.codes

df = df[['text', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


max_sequence_length = 128



class BertLSTMModel(nn.Module):
    def __init__(self, model_name, num_labels, hidden_dim=128, num_layers=2):
        super(BertLSTMModel, self).__init__()
        
        self.bert = AutoModel.from_pretrained(model_name)
        self.lstm = nn.LSTM(input_size=self.bert.config.hidden_size, 
                            hidden_size=hidden_dim, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True)
        self.classifier = nn.Linear(hidden_dim * 2, num_labels)
        self.num_labels = num_labels
    
    def forward(self, input_ids, attention_mask, labels=None):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs.last_hidden_state
        lstm_output, _ = self.lstm(sequence_output)
        lstm_output = lstm_output[:, -1, :]  
        logits = self.classifier(lstm_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
        )



class BertCNNModel(nn.Module):
    def __init__(self, model_name, num_labels, num_filters=100, filter_sizes=[3, 4, 5]):
        super(BertCNNModel, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, self.bert.config.hidden_size)) for fs in filter_sizes
        ])
        self.classifier = nn.Linear(len(filter_sizes) * num_filters, num_labels)
        self.num_labels = num_labels


    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = bert_outputs.last_hidden_state.unsqueeze(1) 

        conv_outputs = [torch.relu(conv(sequence_output)).squeeze(3) for conv in self.convs]
        pooled_outputs = [torch.max(output, 2)[0] for output in conv_outputs]
        cat_output = torch.cat(pooled_outputs, 1)
        
        logits = self.classifier(cat_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
        )



models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
        'UBC-NLP/MARBERT',
        'UBC-NLP/MARBERTv2',
        'faisalq/SaudiBERT',
        'SaudiBERT_LSTM',
        'SaudiBERT_CNN',
    
]


seeds = [0, 1, 42]


for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            if(model_name == 'SaudiBERT_LSTM'):
                tokenizer = AutoTokenizer.from_pretrained('faisalq/SaudiBERT')
                model = BertLSTMModel(model_name='faisalq/SaudiBERT', num_labels=classes_num).to('cuda')       
            elif(model_name == 'SaudiBERT_CNN'):
                tokenizer = AutoTokenizer.from_pretrained('faisalq/SaudiBERT')
                model = BertCNNModel(model_name='faisalq/SaudiBERT', num_labels=classes_num).to('cuda')
            else:
                tokenizer = AutoTokenizer.from_pretrained(model_name)                
                model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['text'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True) 
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)  
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')  
                precision = precision_score(labels, predictions, average='macro')
                recall = recall_score(labels, predictions, average='macro')
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{precision},{recall},{f1}\n')
                return {'accuracy': acc, 'precision': precision, 'recall':recall, 'f1_score': f1}
    
    
            
            
            epochs = 8
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 50, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 50
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'Precision', 'Recall', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2024-09-30 07:03:39.214795: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 07:03:39.237204: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 07:03:39.591913: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


11953

Index(['text', 'sentiment', 'emotion'], dtype='object')

11953

,text,sentiment,emotion
0,الله يعوضك خيرا مماخسرت ويوفقك من خيري الدنيا والاخرة,p,Sadness
1,من جدك هذا وانت استاذنا الله يعوضك اضعافها يارب,p,Sadness
2,الله يعوضك اضعاف ماخسرت ويرزقك بر ولدك وجميع الاخوان,p,Sadness
3,انما النصر صبر ساعه,p,Happiness


...

Some weights of BertModel were not initialized from the model checkpoint at faisalq/SaudiBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/9562 [00:00<?, ? examples/s]

Map:   0%|          | 0/2391 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
50,0.490200,0.275116,0.897951,0.897899,0.895200,0.895709
100,0.286200,0.223632,0.917608,0.919990,0.915330,0.915221
150,0.242600,0.217921,0.911334,0.912974,0.910232,0.909172
200,0.110400,0.212891,0.925972,0.927505,0.923790,0.925140
250,0.105400,0.190162,0.933082,0.931510,0.932637,0.931663
300,0.119300,0.169901,0.932664,0.932799,0.930635,0.931581
350,0.039300,0.283441,0.921790,0.921781,0.923106,0.920445
400,0.078400,0.293198,0.912171,0.919102,0.907620,0.910440
450,0.059700,0.219507,0.931828,0.931562,0.932689,0.931330
500,0.026100,0.295175,0.919281,0.922301,0.916273,0.917645


,Model,Accuracy,Precision,Recall,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.842744,0.840966,0.840906,0.840933
3,SaudiBERT_CNN,0.955249,0.954745,0.954666,0.954687
4,SaudiBERT_LSTM,0.943120,0.943328,0.941571,0.942275
7,UBC-NLP/MARBERT,0.883312,0.881784,0.882449,0.881917
10,UBC-NLP/MARBERTv2,0.897114,0.896127,0.896277,0.896080
13,aubmindlab/bert-base-arabertv02-twitter,0.894187,0.893118,0.893231,0.893167
16,faisalq/SaudiBERT,0.951066,0.951231,0.950583,0.950145
19,qarib/bert-base-qarib,0.897532,0.896203,0.897379,0.896478
